In [1]:
import time
import random
import re

import pandas as pd
from tqdm.auto import tqdm
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [2]:
url = "https://www.wadiz.kr/web/wreward/main"

# Chrome WebDriver 경로 설정
driver = webdriver.Chrome()

# 웹사이트 열기
driver.get(url)

# 페이지 로드를 기다림
driver.implicitly_wait(10)

tabs_mobile_dict = {
    '전체': 1, 'best펀딩': 2, '테크, 가전': 3, 
    '패션': 4, '뷰티': 5, '홈, 리빙': 6,
    '스포츠, 아웃도어': 7, '푸드': 8, '도서, 전자책': 9, 
    '클래스': 10, '디자인': 11, '반려동물': 12, 
    '아트': 13, '캐릭터, 굿즈': 14, '영화, 음악': 15, 
    '키즈': 16, '게임': 17, '만화, 웹툰': 18, 
    '핸드메이드': 19, '사전': 20, '여행': 21, 
    '자동차': 22, '멤버십': 23, '소셜': 24
    }

# XPath를 이용해 요소 찾기
tabs_mobile = tabs_mobile_dict['테크, 가전']
tabs_mobile_element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[4]/div/div/div/ul/li[{tabs_mobile}]/button')
# 요소 클릭
tabs_mobile_element.click()
# time.sleep(1)

# 페이지 로드를 기다림
driver.implicitly_wait(10)

In [3]:
# 드롭다운 요소 식별
dropdown = driver.find_element(By.CLASS_NAME, "OrderSelect_selectBox__2hBjk") # 전체, 진행중, 종료된

# Select 객체 생성
select = Select(dropdown)

# 옵션 선택 방법
dropdown_select_dict = {'0전체': "전체", "1진행중": "진행중", "2종료된": "종료된"}

# 원하는 옵션 선택
select.select_by_visible_text(dropdown_select_dict["1진행중"])

# 페이지 로드를 기다림
driver.implicitly_wait(10)

In [4]:
# 정렬 순서 변경
sort_criteria_dict = {'추천순': 1, '인기순': 2, '모집금액순': 3, '마감임박순': 4, '최신순': 5, '응원참여순': 6}
sort_num = sort_criteria_dict['인기순']
sort_criteria_element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[6]/div[1]/div/ul/li[{sort_num}]')

# 요소 클릭
sort_criteria_element.click()
# time.sleep(1)

# 페이지 로드를 기다림
driver.implicitly_wait(10)


In [5]:
# # 더이상 로딩이 되는 것 없을 때까지 스크롤 다운 반복 - 진짜 긴 것은 어느정도인지 확인용
# last_height = driver.execute_script("return document.body.scrollHeight")

# scroll_count = 0 #스크롤 몇번 했는지 세어주는 것

# while True:
#     # 현재 페이지 기준 끝까지 아래로 스크롤 다운하기
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     # 페이지 로드를 기다림
#     driver.implicitly_wait(10)
#     time.sleep(1)
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         print(f"**break** {scroll_count}번 스크롤링, 최대{48*(scroll_count+1)}개 로딩됨", flush=True)
#         break
    
#     last_height = new_height
#     scroll_count += 1
    
#     # if scroll_count >21:
#     #     print(f"너무 데이터가 많아서 {48*(scroll_count+1)}개까지만 했습니다", flush=True)
#     #     break

In [5]:
# 더이상 로딩이 되는 것 없을 때까지 스크롤 다운 반복
last_height = driver.execute_script("return document.body.scrollHeight")

scroll_count = 0 #스크롤 몇번 했는지 세어주는 것

while True:
    # 현재 페이지 기준 끝까지 아래로 스크롤 다운하기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 페이지 로드를 기다림
    driver.implicitly_wait(10)
    time.sleep(1)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print(f"**break** {scroll_count}번 스크롤링, 최대{48*(scroll_count+1)}개 로딩됨", flush=True)
        break
    
    last_height = new_height
    scroll_count += 1
    
    if scroll_count >21:
        print(f"너무 데이터가 많아서 {48*(scroll_count+1)}개까지만 했습니다", flush=True)
        break

**break** 2번 스크롤링, 최대144개 로딩됨


In [6]:
# 현재 페이지 제일 위로 스크롤 업하기
driver.execute_script("window.scrollTo(document.body.scrollHeight, 0);")

In [11]:
# body = driver.find_element(By.TAG_NAME, "body")
# body.send_keys(Keys.F12)
# body.send_keys(Keys.PAGE_DOWN)

In [9]:
# 총 몇개 로딩되었는지 알아내기 용도
tot_num_for_html = driver.page_source
tot_num_for_soup = BeautifulSoup(tot_num_for_html, 'html.parser')
len(tot_num_for_soup.select(".HomeHorizontalCard_container__QBqLW"))

0

In [10]:
# tot_num_for_soup

<html class="scroll-half-screen" lang="ko"><head>
<style>.async-hide { opacity: 0 !important} </style>
<script async="" src="https://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/signals/config/1129096333844762?v=2.9.154&amp;r=stable&amp;domain=www.wadiz.kr&amp;hme=c3a545c63044e8e9102d4f32d84a1137594d024f28e801d670bc76dc5c075575&amp;ex_m=67%2C112%2C99%2C103%2C58%2C3%2C93%2C66%2C15%2C91%2C84%2C49%2C51%2C158%2C161%2C172%2C168%2C169%2C171%2C28%2C94%2C50%2C73%2C170%2C153%2C156%2C165%2C166%2C173%2C121%2C14%2C48%2C178%2C177%2C123%2C17%2C33%2C38%2C1%2C41%2C62%2C63%2C64%2C68%2C88%2C16%2C13%2C90%2C87%2C86%2C100%2C102%2C37%2C101%2C29%2C25%2C154%2C157%2C130%2C27%2C10%2C11%2C12%2C5%2C6%2C24%2C21%2C22%2C54%2C59%2C61%2C71%2C95%2C26%2C72%2C8%2C7%2C76%2C46%2C20%2C97%2C96%2C9%2C19%2C18%2C81%2C53%2C79%2C32%2C70%2C0%2C89%2C31%2C78%2C83%2C45%2C44%2C82%2C36%2C4%2C85%2C77%2C42%2C39%2C34%2C80%2C2%2C35%2C60%2C40%2C98%2C43%2C75%2C6

In [13]:
# driver.current_url

'https://www.wadiz.kr/web/campaign/detail/162091?acid=10055406&_refer_section_st=REWARD_0'

In [8]:
# def safe_select(soup, number, selector, index=0, value_type="str"):
#     try:
#         results = soup.select(selector)
#         if results is None:
#             return None
#         if results and len(results) > index:
#             if value_type == "num":
#                 digits = re.findall('\d+', results[index].text) 
#                 if results[index].text == "오늘 자정 마감":
#                     print("체크용, 오늘자정마감이라 0으로")
#                     return 0
#                 elif not digits:
#                     return None
#                 else:
#                     return ''.join(digits)
#             else:
#                 return results[index].text
#         else:
#             return None
#     except Exception as e:
#         print(f"{number}번 째 {selector}에서 예외 발생: {e}")
#         return None

# number = 5
# # XPath를 이용해 요소 찾기
# content_element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[6]/div[2]/div[{number}]/a')

# # 컨텐츠 요소 클릭
# content_element.click()
# time.sleep(random.uniform(1,3))
# # 페이지 로드를 기다림
# driver.implicitly_wait(10)

# # 페이지의 HTML 가져오기
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
# safe_select(
#                 soup, number, selector='.BasePreOrderInfo_rate__2uwso span span', index=0, value_type="num")

체크용, 오늘자정마감이라 0으로


0

In [10]:
# 데이터 긁어오기

data = []

# # 총 몇개 로딩되었는지 알아내기 용도
# tot_num_for_html = driver.page_source
# tot_num_for_soup = BeautifulSoup(tot_num_for_html, 'html.parser')

# 기본적으로 로딩 되는 것이 1~48까지 48개 로딩됨, 그리고 추가 로딩 되면 48개씩 추가 로딩됨
expected_end_num = len(tot_num_for_soup.select(".HomeHorizontalCard_container__QBqLW")) # 갯수

# 혹시 잘못 되었을 때 대비해서 안전장치
if expected_end_num == 0:
    print(f"파싱되지 않은 상태라 최대가능갯수로 설정 {expected_end_num} -> {48*(scroll_count+1)}")
    expected_end_num = 48*(scroll_count+1)
    
elif expected_end_num > 48*(scroll_count+1):
    print(f"갯수가 이상합니다, 파싱결과:{expected_end_num} > 최대가능갯수:{48*(scroll_count+1)}")
    print(f"그래서 갯수를 최대가능갯수:{48*(scroll_count+1)}로 변경합니다")
    expected_end_num = 48*(scroll_count+1)

# 찾기 작업을 수행하는 함수
def safe_find(soup, number, tag, class_=None, value_type="str"):
    try:
        result = soup.find(tag, class_=class_)
        if result is None:
            return None
        if value_type == 'num':
            digits = re.findall('\d+', result.text)
            if not digits:
                return None
            else:
                return ''.join(digits)
        else:
            return result.text
    except Exception as e:
        print(f"{number}번 째 {tag}// {class_}에서 예외 발생: {e}")
        return None

# select 메소드를 사용하는 작업을 수행하는 함수
def safe_select(soup, number, selector, index=0, value_type="str"):
    try:
        results = soup.select(selector)
        if results is None:
            return None
        if results and len(results) > index:
            if value_type == "num":
                digits = re.findall('\d+', results[index].text) 
                if results[index].text == "오늘 자정 마감":
                    print("체크용, 오늘자정마감이라 0으로")
                    return 0
                elif not digits:
                    return None
                else:
                    return ''.join(digits)
            else:
                return results[index].text
        else:
            return None
    except Exception as e:
        print(f"{number}번 째 {selector}에서 예외 발생: {e}")
        return None

for number in tqdm(range(1, expected_end_num+1)): #11
    scraped_data = {
        '확인용_num': number,
        'big_tag': None,
        'hash_tag': None,
        'funding_type': None,
        'funding_type_secondary': None,
        'title': None,
        'description': None,
        'num_of_participants': None,
        'left_day': None,
        'amount': None,
        'achieve_rate': None,
        'url': None,
    }
    try:
        # XPath를 이용해 요소 찾기
        content_element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[6]/div[2]/div[{number}]/a')
        
        # 컨텐츠 요소 클릭
        content_element.click()
        time.sleep(random.uniform(1,3))
        # 페이지 로드를 기다림
        driver.implicitly_wait(10)
        
        # 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # print(soup.prettify())
        
        # 프리오더이냐 펀딩이냐에 따라서 태그의 값들이 다른 경우가 있음
        
        # 어느 것인지 확인하는 구분자
        parsing_sep = soup.select_one(".CommonInfo_container__1J2gi").find("div").get("class")[0]
        
        
        # 프리오더/ 펀딩에 따라 태그 다른 값들
        if parsing_sep == "BasePreOrderInfo_container__2TMxc": # 프리오더
            scraped_data['funding_type'] = safe_find(soup, number=number, tag="span", class_="FundingTypeBadge_preorder__20AX-")
            scraped_data['funding_type_secondary'] = safe_find(soup, number, tag="span", class_="Badge_label__3rWfq")
            scraped_data['title'] = safe_find(soup, number, tag="p", class_="BasePreOrderInfo_title__jxsg5")
            scraped_data['description'] = safe_find(soup, number, tag="p", class_="BasePreOrderInfo_description__LxWkO")
            scraped_data['num_of_participants'] = safe_select(
                soup, number, selector='.BasePreOrderInfo_infoWrapper__3wo9k span', index=0, value_type="num")
            scraped_data['left_day'] = safe_select(
                soup, number, selector='.BasePreOrderInfo_rate__2uwso span span', index=0, value_type="num")
            scraped_data['amount'] = safe_find(soup, number, tag="p", class_="BasePreOrderInfo_totalBackedAmount__2FNmg", value_type="num")
            scraped_data['achieve_rate'] = safe_select(
                soup, number, selector='.BasePreOrderInfo_rate__2uwso span span', index=1, value_type="num")
        
        elif parsing_sep == "BaseFundingInfo_container__sotrP": #펀딩
            scraped_data['funding_type'] = safe_find(soup, number, tag="span", class_="FundingTypeBadge_funding__4BhpK")
            scraped_data['funding_type_secondary'] = "없음" # None
            scraped_data['title'] = safe_find(soup, number, tag='p', class_="BaseFundingInfo_title__1Dw42")
            scraped_data['description'] = safe_find(soup, number, tag="p", class_="BaseFundingInfo_description__1XXyy")
            scraped_data['achieve_rate'] = safe_select(
                soup, number, selector='.BaseFundingInfo_infoWrapper__23LoT span', index=0, value_type="num")
            scraped_data['left_day'] = safe_select(
                soup, number, selector='.BaseFundingInfo_rate__1uMoD span span', index=0, value_type="num")
            scraped_data['amount'] = safe_find(soup, number, tag="p", class_="BaseFundingInfo_totalBackedAmount__1ezH_", value_type="num")
            scraped_data['num_of_participants'] = safe_select(
                soup, number, selector='.BaseFundingInfo_rate__1uMoD span span', index=1, value_type="num")
            
        
        else:
            raise ValueError(f"확인한 것과 다른 구분자가 확인되었습니다 : {parsing_sep}")
        # 공통적인 것, 예외처리 안해줬음
        scraped_data['big_tag'] = soup.find('span', {'aria-label': '카테고리'}).text
        scraped_data['hash_tag'] = soup.select('.ProjectSearchTags_hashTag__2UMo_ span')[1].text
        scraped_data['url'] = driver.current_url
        
        # # 오늘 자정 마감일 경우에
        # if scraped_data['left_day'] is None:
        #      scraped_data['left_day'] = 0
        
        data.append(scraped_data)
        
        # 되돌아가기
        driver.back()
        time.sleep(random.uniform(1,3))
        # 페이지 로드를 기다림
        driver.implicitly_wait(10)
        
    except NoSuchElementException:
        print(f"요소를 찾을 수 없습니다. {number}를 찾지 못했습니다") #, flush=True
        break  # 요소를 찾지 못하면 for문 중단
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")
        print(f"{number}째에서 발생했습니다: {e}")
        continue  # 다음 항목으로 계속 진행

파싱되지 않은 상태라 최대가능갯수로 설정 0 -> 144


  0%|          | 0/144 [00:00<?, ?it/s]

체크용, 오늘자정마감이라 0으로
예상치 못한 오류가 발생했습니다: 'NoneType' object has no attribute 'find'
6째에서 발생했습니다: 'NoneType' object has no attribute 'find'
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
체크용, 오늘자정마감이라 0으로
요소를 찾을 수 없습니다. 123를 찾지 못했습니다


In [11]:
# WebDriver 종료
driver.quit()

In [12]:
# 데이터 프레임 생성
df_sights = pd.DataFrame(data)

# 데이터 프레임 출력
df_sights

,확인용_num,big_tag,hash_tag,funding_type,funding_type_secondary,title,description,num_of_participants,left_day,amount,achieve_rate,url
0,1,생활가전,청소,펀딩,없음,[먼지킬러] 분당 9000번! 강력진동으로 먼지를 제거합니다! 제트브러시,봄맞이 대청소! 청소혁명! 집안 구석구석 쌓인 먼지를 한번에 싹! 세계최초 3차원 ...,598,17,39745010,3974,https://www.wadiz.kr/web/campaign/detail/16209...
1,2,주변기기,테크,프리오더-글로벌,국내단독,"[글로벌10억] 접으면 무선 마우스, 펼치면 어디든 함께! 외근 필수템","[일본미국 10억 펀딩] 장담해요. 이제 더 이상 플라스틱 마우스를 떨어뜨릴 일도,...",4126,27,342496401,34249,https://www.wadiz.kr/web/campaign/detail/27695...
2,3,엔터테인먼트가전,선물,펀딩,없음,"[CES2024] 착-붙 메모지 프린터, GPT를 만나 더욱 똑똑해졌어요","세상에서 가장 똑똑한 미니 프린터 네모닉AI!\n누적 200억, CES 최고혁신상,...",1655,25,130455500,13045,https://www.wadiz.kr/web/campaign/detail/27556...
3,4,생활가전,선풍기,프리오더,특별구성,1~100단까지 진짜 강력한 바람 등장! 파워 휴대용 에어컨 선풍기,"와디즈 1위 휴대용 선풍기, 에어컨 선풍기가 바람이 더 강력해져 돌아왔습니다.\n1...",1042,14,49846900,9969,https://www.wadiz.kr/web/campaign/detail/27665...
4,5,웨어러블,충전기,프리오더-앵콜,특별구성,2만원대 넣을수 없는 기능을 모두 담았습니다｜맥세이프 보조배터리혁명,무겁고 투박한 보조배터리는 이제 그만! 어디에서도 볼 수 없는 유니크한 디자인과 2...,970,0,34617000,6923,https://www.wadiz.kr/web/campaign/detail/27668...
...,...,...,...,...,...,...,...,...,...,...,...,...
116,118,App·Web,주식,프리오더-앵콜,국내단독,[2차앵콜]투자가 쉬워진다!하루5분으로 쉽게 관리하는 AI재테크 솔루션,"주식, 힘들고 막막하셨다구요? 알파시스템이 도와드리겠습니다!\n미국, 기관에서는 이...",1,34,500000,5,https://www.wadiz.kr/web/campaign/detail/28130...
117,119,주변기기,맥세이프,프리오더-앵콜,특별구성,맥세이프 자석 기능 탑재! 메탈 마우스 패드 : 시원한 사용 경험!,튼튼한 메탈 소재로 제작되어 내구성이 뛰어나며 360도 회전이 가능한 맥세이프 자석...,1,25,52900,10,https://www.wadiz.kr/web/campaign/detail/28458...
118,120,주방가전,드립커피,프리오더-앵콜,국내단독,업그레이드된 모터로 성능이 업그레이드되어 돌아온 홈카페 그라인더 끝판왕!,"모터 정밀기어박스 업그레이드로 성능은 더 강력해졌지만, 가격은 더 착해진 58esp...",0,6,0,0,https://www.wadiz.kr/web/campaign/detail/27912...
119,121,생활가전,캠핑,프리오더-글로벌,국내단독,발받침대 기능 총집합. 바른자세와 편안함을 동시에 스마트 발 받침대!,기존의 발 받침대들의 기능들을 통폐합했습니다!높이 조절 가능한 발바닥 및 푹신한 다...,0,20,0,0,https://www.wadiz.kr/web/campaign/detail/28238...


In [27]:
df_sights[df_sights['left_day']==0]

,확인용_num,big_tag,hash_tag,funding_type,funding_type_secondary,title,description,num_of_participants,left_day,amount,achieve_rate,url


In [38]:
# col 생략 없이 출력
pd.set_option('display.max_rows', 200)

In [36]:
df_sights.isna().sum()

확인용_num                    0
big_tag                    0
hash_tag                   0
funding_type               0
funding_type_secondary     0
title                      0
description                0
num_of_participants        0
left_day                  11
amount                     0
achieve_rate               0
url                        0
dtype: int64

In [34]:
df_sights[df_sights['left_day'].isna()]

,확인용_num,big_tag,hash_tag,funding_type,funding_type_secondary,title,description,num_of_participants,left_day,amount,achieve_rate,url
4,5,웨어러블,충전기,프리오더-앵콜,특별구성,2만원대 넣을수 없는 기능을 모두 담았습니다｜맥세이프 보조배터리혁명,무겁고 투박한 보조배터리는 이제 그만! 어디에서도 볼 수 없는 유니크한 디자인과 2...,940,None,33552300,6710,https://www.wadiz.kr/web/campaign/detail/27668...
10,11,생활가전,캠핑용품,프리오더-앵콜,국내단독,"텀블러 만한 사이즈로 건조,살균,탈취까지? 실속만 남긴 휴대용 옷 건조기","온가담 휴대용 옷건조기만 있다면 여행,출장지 빨래 걱정끝!\n여행지,출장지,집에서 ...",285,None,13496900,2699,https://www.wadiz.kr/web/campaign/detail/27872...
16,17,생활가전,선풍기,프리오더,특별구성,"[한정수량] 선풍기 근본 신일전자｜스펙은 평균이상, 안전까지 생각했습니다",신일 프리미엄 2in1 DC팬!\n더 가볍고 더 조용하게 특허 받은 DC팬 선풍기!...,207,None,25959000,2595,https://www.wadiz.kr/web/campaign/detail/27246...
24,25,생활가전,캠핑,프리오더-앵콜,국내단독,[앵콜]갓성비 분위기 깡패!3단계 빛과 바람 다재다능 무선 캠핑 선풍기,눈부신 조명은 안녕~ 강력한 풍량은 기본! 바이니만의 선풍기 하단 전용 랜턴으로 감...,148,None,8563800,1712,https://www.wadiz.kr/web/campaign/detail/25409...
31,32,주변기기,휴대폰거치대,프리오더,특별구성,[와디즈 단독 출시] 여행의 필수품 폴더형 맥세이프 스마트폰 거치대,이번에 소개드릴 제품은 2024년 폴더형 맥세이프 거치대입니다. 최근 해외여행 급증...,102,None,3226000,645,https://www.wadiz.kr/web/campaign/detail/27751...
32,33,욕실,피부,프리오더,특별구성,[피부임상완료] 2중 필터로 관리하는 퓨리풀 피부케어 샤워기,샤워기가 피부 임상시험을 받았다고? YES! 2중 멤브레인 필터로 깨끗하게 정수된 ...,86,None,2334600,466,https://www.wadiz.kr/web/campaign/detail/25989...
37,38,테크·가전,가방,펀딩,없음,20만원대 브람스 안마의자? 100만원대 안마의자 기술을 압축했습니다.,안마 브랜드의 뿌리 깊은 역사를 담고 있는 브람스의 역작. 100만 원 대 안마의자...,62,None,16986000,1698,https://www.wadiz.kr/web/campaign/detail/27428...
48,49,주방가전,반려동물,펀딩,없음,[세상에서 가장 작은 화식 공장] 10분이면 자연식 한 끼 뚝딱! 펫쿠커,똑같은 사료를 먹는 우리 댕냥이에게 맛있고 싱싱한 자연식을 직접 식재료를 선택하는 ...,33,None,2790000,279,https://www.wadiz.kr/web/campaign/detail/27244...
57,58,생활가전,귀지청소,프리오더-앵콜,특별구성,[앵콜특별가] 귀 속을 직접 보고 관리하자! 아우라 LED내시경 귀클리너,"직접 보고 관리할 수 있는 귀 내시경! LED조명 탑재!\n왼쪽,오른쪽 구분 뿐 아...",28,None,673400,134,https://www.wadiz.kr/web/campaign/detail/27921...
58,59,뷰티디바이스,다이어트,프리오더,특별구성,"i 후기인증 i 셀룰라이트, 바디라인 정리에 지방을 녹이는 '순수'고주파",[N회차 앵콜] 셀룰라이트 관리/바디 라인 탄력/근육 뭉침까지 3주마다 달라지는 ...,27,None,4482000,896,https://www.wadiz.kr/web/campaign/detail/28136...


In [14]:
df_sights.iloc[4]['left_day']

AttributeError: 'int' object has no attribute 'dtype'

In [23]:
df_sights.iloc[4]['url']

'https://www.wadiz.kr/web/campaign/detail/276680?_refer_section_st=PREORDER_4'

In [35]:
df_sights.loc[df_sights['left_day']=='1', 'url'].to_list()

['https://www.wadiz.kr/web/campaign/detail/276107?_refer_section_st=PREORDER_39',
 'https://www.wadiz.kr/web/campaign/detail/278690?_refer_section_st=PREORDER_47',
 'https://www.wadiz.kr/web/campaign/detail/284101?_refer_section_st=PREORDER_59',
 'https://www.wadiz.kr/web/campaign/detail/282985?_refer_section_st=PREORDER_68',
 'https://www.wadiz.kr/web/campaign/detail/276804?_refer_section_st=PREORDER_72',
 'https://www.wadiz.kr/web/campaign/detail/276441?_refer_section_st=PREORDER_76',
 'https://www.wadiz.kr/web/campaign/detail/281073?_refer_section_st=PREORDER_79',
 'https://www.wadiz.kr/web/campaign/detail/278139?_refer_section_st=PREORDER_85',
 'https://www.wadiz.kr/web/campaign/detail/277634?_refer_section_st=PREORDER_90']

In [16]:
df_sights.iloc[[15,16,31,32]]

,big_tag,hash_tag,funding_type,funding_type_secondary,title,description,num_of_participants,left_day,amount,achieve_rate,url,확인용_num
15,스마트가전,잠,펀딩,없음,"5cm 우리집 작은 주치의, AI 수면 측정기기 [좋은 잠, 웰룬]","이제, 꿀.잠.길 걸으세요! 매일매일 나의 수면 패턴을 체크하고, 모니터링 해요. ...",234,20,15413000,1541,https://www.wadiz.kr/web/campaign/detail/26519...,16
16,테크·가전,외국인선물,프리오더,특별구성,[AI번역기]난 해외에서도 한국말 쓴다! 내 손안의 작은 통역사 우애스크,4개의 글로벌기업 번역엔진을 사용하여 144개 언어를 지원하는 유일한 AI동시통역기!,214,1,13419400,2683,https://www.wadiz.kr/web/campaign/detail/27890...,18
31,생활가전,미니글루건,프리오더,국내단독,[생활필수품] 디지털 온도도절이 가능한 무선 글루건+C타입 충전,150~200℃ 온도조절이 가능한 40W 무선글루건. 글루가 분당 16g이나 토출되...,113,10,6091500,609,https://www.wadiz.kr/web/campaign/detail/27424...,33
32,엔터테인먼트가전,빔,프리오더-앵콜,특별구성,"[누적 3억]20만원 대 FHD화질 미친스펙, 제우스 빔프로젝터니까!","1000안시루멘 / 200,000:1 동적명암비 / 30000시간 램프수명 / Fu...",110,4,16322000,1632,https://www.wadiz.kr/web/campaign/detail/27885...,35


In [14]:
import datetime

datetime.datetime.today()

datetime.datetime(2024, 4, 22, 15, 38, 19, 421949)

In [15]:
datetime_info = datetime.datetime.today().strftime("%Y%m%d_%H%M")

In [18]:
df_sights = df_sights.astype({'num_of_participants': 'int32', 'left_day': 'int8', 'amount': 'int32', 'achieve_rate': 'int32'})

In [19]:
df_sights.to_parquet(f"./크롤링/{datetime_info}테크가전.parquet")

# 추가 개선 확인부분

In [8]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

url = "https://www.wadiz.kr/web/wreward/main"

# Chrome WebDriver 경로 설정
driver = webdriver.Chrome()

# 웹사이트 열기
driver.get(url)

# 페이지 로드를 기다림
driver.implicitly_wait(10)

# XPath를 이용해 요소 찾기
category = 3
element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[4]/div/div/div/ul/li[{category}]/button')
# 요소 클릭
element.click()
time.sleep(1)

# XPath를 이용해 요소 찾기
number = 1
element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[6]/div[2]/div[{number}]/a')
# 요소 클릭
element.click()
time.sleep(1)
# 페이지 로드를 기다림
driver.implicitly_wait(10)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [22]:
# 찾기 작업을 수행하는 함수
def safe_find(soup, number, tag, class_=None):
    try:
        result = soup.find(tag, class_=class_)
        return ''.join(re.findall('\d+', result.text)) if result else None
    except Exception as e:
        print(f"{number}번 째 {tag}// {class_}에서 예외 발생: {e}")
        return None

# select 메소드를 사용하는 작업을 수행하는 함수
def safe_select(soup, number, selector, index=0):
    try:
        results = soup.select(selector)
        if results and len(results) > index:
            return ''.join(re.findall('\d+', results[index].text))
        else:
            return None
    except Exception as e:
        print(f"{number}번 째 {selector}에서 예외 발생: {e}")
        return None

In [12]:
safe_find(soup, number, tag="p", class_="BaseFundingInfo_totalBackedAmount__1ezH_")

'37,171,010원 달성'